http request pollyfill?

how to https request?


In [ ]:
var promisify
try {
    promisify = require('util').promisify
} catch (e) {
    if(!e.message.includes('Cannot find module')) {
        throw e
    }
}
if(!promisify) {
    try {
        promisify = require('bluebird').promisify
    } catch (e) {
        if(!e.message.includes('Cannot find module')) {
            throw e
        }
    }    
}

try {
    var httpRequest = util.promisify(require('request'));
} catch (e) {
    if(!e.message.includes('Cannot find module')) {
        throw e
    }
    
    var http = require('http')
    var https = require('https')
    var url = require('url')

    function httpRequest(params) {
        var protocol = params.url.includes('http://') ? http : https
        var location = url.parse(params.url)
        var postData = querystring.stringify(params.data);

        Object.assign(params, {
            path: location.pathname,
            host: location.hostname,
            port: location.port || location.protocol === 'http' ? 80 : 443,
            headers: Object.assign(params.headers || {}, {
                'Content-Type': 'application/x-www-form-urlencoded',
                'Content-Length': Buffer.byteLength(postData)
            }, params.headers)
        }, params)

        return new Promise(function(resolve, reject) {
            var req = protocol.request(params, function(res) {
                // cumulate data
                res.setEncoding('utf8');                
                res.body = [];
                res.on('data', function(chunk) {
                    res.body.push(chunk);
                });
                // resolve on end
                res.on('end', function() {
                    try {
                        res.body = res.body.join('');
                    } catch(e) {
                        reject(e);
                    }
                    resolve(res);
                });
            });
            req.on('error', err => reject(err));
            if (params.data) {
                req.write(postData);
            }
            // IMPORTANT
            req.end();
        });
    }
}

function request(...args) {
    return httpRequest(...args)
        .then(res => {
            if(res.statusCode < 200 || res.statusCode >= 300) {
                throw new Error('Error: ' + res.statusCode + ' - ' + JSON.stringify(res.body));
            }
            var isJson = (res.headers['content-type'] || '').match(/application\/json/ig);
            var isPlain = (res.headers['content-type'] || '').match(/text\//ig);
            if(isJson && typeof res.body !== 'undefined') {
                res.body = JSON.parse(res.body.toString('utf8'));
            }
            if(isPlain && typeof res.body !== 'undefined') {
                res.body = res.body.toString('utf8');
            }
            return res;
        })
}

module.exports = {
    request
}
